# Рекомендация тарифов

**Цели и задачи:**

- Построить модель для задачи классификации, которая выберет подходящий для пользователей новый тариф сотовой связи.
- Оптимизировать модель по метрике accuracy (минимально допустимое значение 0.75).
- Проверить accuracy на тестовой выборке.

**План выполнения работы:**

- 1. Изучение общей информации о данных
- 1. Построение моделей классификации
     - 2.1 Решающее дерево
     - 2.2 Случайный лес
     - 2.3 Логистическая регрессия
     - 2.4 К-Ближайших соседей
     - 2.5 Метод Опорных Векторов
     - 2.6 Проверка на тестовой выборке
- 1. Проверка вменяемости модели
- 1. Общий вывод

**Описание данных:**
Каждый объект в наборе данных `users_behavior.csv` — это информация о поведении одного пользователя за месяц.

- `сalls` — количество звонков,
- `minutes` — суммарная длительность звонков в минутах,
- `messages` — количество sms-сообщений,
- `mb_used` — израсходованный интернет-трафик в Мб,
- `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Откройте и изучите файл

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report
import numpy as np
import random

In [19]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [20]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


### Вывод

Файл содержит 3214 строк, 5 столбцов.  
4 столбца имеют вещественный тип данных, 1 - целочисленный.  
Пропущенных значений нет.  
Объем данных порядка 126 КБ.

## Разбейте данные на выборки

In [22]:
df_train, df_valid = train_test_split(df, test_size=0.25, random_state=12345)
df_valid, df_test = train_test_split(df_valid,test_size = 0.5, random_state=12345)

In [23]:
df_train.shape, df_valid.shape, df_test.shape

((2410, 5), (402, 5), (402, 5))

<b>Обучающая выборка</b>

In [24]:
features_train = df_train.drop('is_ultra', axis=1)
target_train = df_train['is_ultra']

<b>Валидационная выборка</b>

In [25]:
features_valid = df_valid.drop('is_ultra', axis=1)
target_valid = df_valid['is_ultra']

<b>Тестовая выборка</b>

In [26]:
features_test = df_test.drop('is_ultra', axis=1)
target_test = df_test['is_ultra']

## Исследуйте модели

### Модель: Решающее дерево

In [27]:
best_model_tree = None
best_result_tree = 0
best_depth_tree = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = model.score(features_valid, target_valid)
    if result > best_result_tree:
        best_model_tree = model
        best_result_tree = result
        best_depth_tree = depth
print('Глубина наилучшей модели:', best_depth_tree)
print('Accuracy наилучшей модели на валидационной выборке:', best_result_tree)

Глубина наилучшей модели: 3
Accuracy наилучшей модели на валидационной выборке: 0.7985074626865671


### Модель: Случайный лес

In [28]:
best_model_forest = None
best_result_forest = 0
best_depth_forest = 0
best_est = 0
for est in range(10, 51, 10):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        result = model.score(features_valid, target_valid)
    if result > best_result_forest:
        best_model_forest = model
        best_result_forest = result
        best_est = est
        best_depth_forest = depth
print('Количество дереьев наилучшей модели:', best_est)
print('Глубина наилучшей модели:', best_depth_forest)
print('Accuracy наилучшей модели на валидационной выборке:', best_result_forest)

Количество дереьев наилучшей модели: 50
Глубина наилучшей модели: 10
Accuracy наилучшей модели на валидационной выборке: 0.8009950248756219


### Модель: Логистическая регрессия

In [29]:
model_logic = LogisticRegression(random_state=12345, solver='lbfgs')
model_logic.fit(features_train, target_train)
result_logic = model_logic.score(features_valid, target_valid)

print('Accuracy модели логистической регрессии на валидационной выборке:', result_logic)

Accuracy модели логистической регрессии на валидационной выборке: 0.7039800995024875


### Вывод

Наибольшая точность у модели Случайный лес (порядка 0.8) с количеством деревьев, равным 50, и глубиной, равной 10.

## Проверьте модель на тестовой выборке

In [30]:
predictions_test = best_model_forest.predict(features_test)

In [31]:
print("Тестовая выборка:", accuracy_score(target_test, predictions_test))

Тестовая выборка: 0.7935323383084577


### Вывод

Точность тестовой выборки для модели Случайный лес составила порядка 0.79.

## (бонус) Проверьте модели на адекватность

Для проверки адекватности модели предположим, что количество абонентов, перешедших на тарифы "Смарт" и "Ультра", пропорционально их количеству в исходных данных.

In [32]:
df['is_ultra'].value_counts(normalize=True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

Видим, что количество абонентов, перешедших на тариф "Смарт", к общему количеству абонентов, составляет порядка 69%.  
Доля правильных ответов модели составляет порядка 79%, что выше нашего предположения (69%). Поэтому, модель будем считать адекватной.

## Вывод

Исходные данные поделены на три выборки: обучающую, валидационную и тестовую.  
В качестве моделей использовались Дерево решений, Случайный лес и Логистическая регрессия. Точность модели Случайный лес оказалась несколько выше, чем точность модели Дерево решений, и составила порядка 80% Точность модели Логистическая регрессия составила порядка 70%. Гиперпараметры модели Случайный лес менялись в цикле, достаточная точность была достигнута при количестве деревьев 50 и глубине 10.  
Тестировани модели показало ее точность, равную 79%.  
Модель также была протестирована на адекватность, результаты оказались положительными.